In [2]:
import numpy as np
import pandas as pd

In [3]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


In [4]:
df = pd.read_csv('1.csv')
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

In [5]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))
df['addr'] = df['neighbourhood'] +' '+ df['location']
df.set_index('name', inplace=True)
indices = pd.Series(df.index)

In [6]:
df

,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
Spice Elephant,4.1,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",Chinese North Indian Thai,800.0,Banashankari,Banashankari Banashankari
San Churro Cafe,3.8,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",Cafe Mexican Italian,800.0,Banashankari,Banashankari Banashankari
Addhuri Udupi Bhojana,3.7,Banashankari,Masala Dosa,South Indian North Indian,300.0,Banashankari,Banashankari Banashankari
Grand Village,3.8,Basavanagudi,"Panipuri, Gol Gappe",North Indian Rajasthani,600.0,Banashankari,Banashankari Basavanagudi
...,...,...,...,...,...,...,...
Chime - Sheraton Grand Bengaluru Whitefield Hotel &...,4.3,"ITPL Main Road, Whitefield","Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
The Nest - The Den Bengaluru,3.4,"ITPL Main Road, Whitefield",NaN,Finger Food North Indian Continental,1500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
Nawabs Empire,3.2,"ITPL Main Road, Whitefield",NaN,North Indian Chinese Arabian Momos,300.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"


In [126]:
l = [i for i in df['neighbourhood']]
l2 = [i for i in df['cuisines']]
l3 = [i for i in df['dish_liked']]

In [8]:
dish=[]
for i in l3:
    if pd.isna(i):
        dish.append('')
    else:
        dish.append(''.join(i.split(',')))

In [127]:
# integer encode the documents
vocab_size = 200
encoded_docs = [one_hot(d, vocab_size) for d in l]
encoded_docs2 = [one_hot(d, vocab_size) for d in l2]
encoded_docs3 = [one_hot(d, vocab_size) for d in dish]

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
padded_docs3 = pad_sequences(encoded_docs3, maxlen=max_length, padding='post')


In [114]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 10, 25)            5000      
_________________________________________________________________
flatten_7 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 251       
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
_________________________________________________________________
None


In [128]:
# fit the model
embd = model.predict(padded_docs)
embd2 = model.predict(padded_docs2)
embd3 = model.predict(padded_docs3)

In [129]:
embd,embd2,embd3

(array([[0.5020615 ],
        [0.5020615 ],
        [0.5020615 ],
        ...,
        [0.50150096],
        [0.50150096],
        [0.50150096]], dtype=float32),
 array([[0.50851893],
        [0.49929962],
        [0.50316423],
        ...,
        [0.5097227 ],
        [0.507089  ],
        [0.50224787]], dtype=float32),
 array([[0.51403624],
        [0.49321172],
        [0.4972608 ],
        ...,
        [0.50013834],
        [0.50013834],
        [0.50013834]], dtype=float32))

In [130]:
sim=[]
for i in range(indices.shape[0]):
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]])
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]]) 
    sim.append([embd[i][0],embd2[i][0],embd3[i][0]])

In [131]:
sim[0]

[0.5020615, 0.50851893, 0.51403624]

In [140]:
def rec(name):
    idx = indices[indices == name].index[0]
    corr,reco = [],[]
    for i in range(len(sim)):
        #corr.append( (cosine_similarity([sim[idx]],[sim[i]])[0][0], i))
        cs = cosine_similarity([sim[idx]],[sim[i]])[0][0]
        if cs>=0.99997:
            corr.append((cs,i))
     
    corr = sorted(corr,key = lambda x:x[0], reverse=True)
    reco = [c[1] for c in corr]
    if idx in reco:reco.remove(idx)
    print('Coverage = ',(len(corr)*100/df.shape[0]),'Percent')
    
    return df.iloc[reco[:11]]
    

    
rec("Jalsa")

Coverage =  7.855191256830601 Percent


,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Sheesh Kebab,4.1,Frazer Town,"Naan, Mutton Kebab, Memon Mutton Biryani, Mutt...",North Indian,700.0,Brigade Road,Brigade Road Frazer Town
Amritsari Kulcha Land,4.1,JP Nagar,"Lassi, Amritsari Kulcha, Punjabi Kulcha, Panee...",North Indian,300.0,Bannerghatta Road,Bannerghatta Road JP Nagar
Samovar,3.9,Kammanahalli,"Paratha, Kerala Parotta, Fish Fry, Sea Food, Tea",Kerala South Indian,600.0,Frazer Town,Frazer Town Kammanahalli
Goa 0 Km,3.6,Kumaraswamy Layout,"Sea Food, Crab Meat, Crab Soup, Fish Curry, Ch...",Goan Seafood North Indian Chinese Biryani,800.0,Banashankari,Banashankari Kumaraswamy Layout
Hyderabad Biryaani Home,3.0,South Bangalore,"Chicken, Hyderabadi Biryani, Mutton Biryani",North Indian Hyderabadi Chinese,650.0,Bellandur,Bellandur South Bangalore
Foodzoo,3.7,HSR,"Paratha, Puri Sabzi, Tea, Chole",North Indian Chinese Street Food,300.0,Bellandur,Bellandur HSR
Magic Oven,4.0,Electronic City,"Sandwiches, Burgers, Coffee, Ginger Tea, Red V...",Cafe Desserts,400.0,Electronic City,Electronic City Electronic City
Udupi Aatithya,4.0,Bellandur,"Filter Coffee, Vada, Biryani, Masala Dosa",South Indian North Indian Chinese,300.0,Bellandur,Bellandur Bellandur
Mango Tree- The Beer Garden,4.1,JP Nagar,"Dragon Chicken, Mutton Biryani, Mocktails, Dal...",North Indian Chinese Continental,1200.0,Bannerghatta Road,Bannerghatta Road JP Nagar


In [134]:
cosine_similarity([sim[1]],[sim[0]])[0][0]

0.99985695